# AO3 query search

A notebook to use scrape metadata from the fanwork archive AO3 and store it in mySQL files. 

In [1]:
# import specialised functions from ao3.py file
%load_ext autoreload
%autoreload 1
%aimport ao3

In [35]:
# necessary libraries. One day I will use beautiful soup.
import requests
import os
import time
import datetime
#from datetime import datetime
#from lxml import html

from bs4 import BeautifulSoup
import string

from collections import OrderedDict
from collections import Counter
from collections import deque

import math
import sqlite3
import pandas as pd
import numpy as np



Initiate the database for query 


In [15]:
url = 'https://archiveofourown.org/tags/Ronnie%20Lee/works'
dbName = 'test_ronnie'

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}
root_url = 'https://archiveofourown.org'

# Start the database
df = pd.DataFrame()
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')
works = soup.find_all(role="article")
for fic in works:
    ent = ao3.query_AO3_work(fic)
    df=df.append(ent,ignore_index=True)
    
u = soup.find_all(rel="next")
if len(u)>0:
    v = BeautifulSoup(str(u[0]),'html.parser')
    NEXTurl = root_url + str(v.a.get('href'))
    print(NEXTurl)
    NextPage = True

ficCount = len(df)
df.loc[ficCount-5:]

https://archiveofourown.org/tags/Ronnie%20Lee/works?page=2


,Bookmarks,Category,Chapters,Character_tags,Collection,Comments,Complete,Creator,Date,Fandom,...,Series,Series_part,Summary,Tags,Title,Warning,Words,cURL,sURL,url
15,1.0,"[Gen, F/F]","[6, 6]","[Ronnie Lee, Ray Butani, Patrick Brewer, The F...",1.0,43.0,Complete Work,"[Amanita_Fierce, DoubleL27]",20201213.0,[Schitt's Creek],...,[],[],"\n“Hello, Ronnie!” Ray yelled, slipping out of...","[Ray Butani & Ronnie Lee, Patrick Brewer & Ron...",Come In From The Cold,[No Archive Warnings Apply],14977.0,[https://archiveofourown.org/users/Amanita_Fie...,[],https://archiveofourown.org/works/27769942
16,16.0,[M/M],"[8, 8]","[David Rose (Schitt's Creek), Patrick Brewer, ...",1.0,29.0,Complete Work,[perfect-porcelain (tedddylupin)],20201212.0,[Schitt's Creek],...,[],[],David Rose is a washed up childhood actor who ...,"[Patrick Brewer/David Rose, David Rose (Schitt...",the christmas chateau,[No Archive Warnings Apply],19092.0,[https://archiveofourown.org/users/tedddylupin...,[],https://archiveofourown.org/works/27750664
17,14.0,[No category],"[5, 5]","[Patrick Brewer, David Rose (Schitt's Creek), ...",0.0,32.0,Complete Work,[writemenot],20201210.0,[Schitt's Creek],...,[],[],"\n“I’m really glad you stole my boss’s dinner,...","[Patrick Brewer/David Rose, Patrick Brewer, Da...",you got my future (in your hands),[Choose Not To Use Archive Warnings],26578.0,[https://archiveofourown.org/users/writemenot/...,[],https://archiveofourown.org/works/27570559
18,36.0,[M/M],"[1, 1]","[David Rose (Schitt's Creek), Patrick Brewer, ...",1.0,82.0,Complete Work,[this_is_not_nothing],20201210.0,[Schitt's Creek],...,[],[],"For the prompt: ""Who doesn’t love a wintry or ...","[Patrick Brewer/David Rose, David Rose (Schitt...",How Sweet It Is,[No Archive Warnings Apply],11802.0,[https://archiveofourown.org/users/this_is_not...,[],https://archiveofourown.org/works/27758566
19,8.0,[M/M],"[1, 1]","[Patrick Brewer, David Rose (Schitt's Creek), ...",1.0,64.0,Complete Work,[mycleverusername],20201208.0,[Schitt's Creek],...,[],[],"\n“David, I’ll have you know that I take gift ...","[Patrick Brewer/David Rose, Patrick Brewer, Da...",present company excepted,[No Archive Warnings Apply],3550.0,[https://archiveofourown.org/users/mycleveruse...,[],https://archiveofourown.org/works/27765349


Continue to scrape subsequent pages of query into database (pandas dataframe.)

In [16]:
while NextPage:
    time.sleep(5)
    page = requests.get(NEXTurl, headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    works = soup.find_all(role="article")
    for fic in works:
        ent = ao3.query_AO3_work(fic)
        df=df.append(ent,ignore_index=True)

    u = soup.find_all(rel="next")
    if len(u)>0:
        v = BeautifulSoup(str(u[0]),'html.parser')
        NEXTurl = root_url + str(v.a.get('href'))
        print(NEXTurl)
        NextPage = True
    else:
        NextPage = False

ficCount = len(df)
df.loc[ficCount-3:]

https://archiveofourown.org/tags/Ronnie%20Lee/works?page=3
https://archiveofourown.org/tags/Ronnie%20Lee/works?page=4
https://archiveofourown.org/tags/Ronnie%20Lee/works?page=5
https://archiveofourown.org/tags/Ronnie%20Lee/works?page=6
https://archiveofourown.org/tags/Ronnie%20Lee/works?page=7
https://archiveofourown.org/tags/Ronnie%20Lee/works?page=8
https://archiveofourown.org/tags/Ronnie%20Lee/works?page=9
https://archiveofourown.org/tags/Ronnie%20Lee/works?page=10
https://archiveofourown.org/tags/Ronnie%20Lee/works?page=11
https://archiveofourown.org/tags/Ronnie%20Lee/works?page=12


,Bookmarks,Category,Chapters,Character_tags,Collection,Comments,Complete,Creator,Date,Fandom,...,Series,Series_part,Summary,Tags,Title,Warning,Words,cURL,sURL,url
227,54.0,[M/M],"[1, 1]","[David Rose (Schitt's Creek), Patrick Brewer, ...",0.0,13.0,Complete Work,[mihaly],20181207.0,[Schitt's Creek],...,[],[],Patrick wants to move their relationship forwa...,"[Patrick Brewer/David Rose, David Rose (Schitt...",no need to worry,[No Archive Warnings Apply],4000.0,[https://archiveofourown.org/users/mihaly/pseu...,[],https://archiveofourown.org/works/16895190
228,143.0,[M/M],"[1, 1]","[David Rose, Patrick Brewer, Stevie Budd, Moir...",0.0,57.0,Complete Work,[falconeggs],20181129.0,[Schitt's Creek],...,[],[],“Patrick loves David the exact same way Johnny...,"[Patrick Brewer/David Rose, Minor Johnny Rose/...",Special Order Forms,[No Archive Warnings Apply],10164.0,[https://archiveofourown.org/users/falconeggs/...,[],https://archiveofourown.org/works/16782049
229,4.0,"[F/M, M/M]","[5, 5]","[Johnny Rose, Moira Rose, Original Male Charac...",0.0,9.0,Complete Work,[Miso],20180308.0,[Schitt's Creek],...,[],[],Johnny's life is going fine. Until a certain s...,"[Johnny Rose/Moira Rose, Johnny Rose/Original ...",Without A Paddle,[No Archive Warnings Apply],3561.0,[https://archiveofourown.org/users/Miso/pseuds...,[],https://archiveofourown.org/works/13910523


Comb through database and drop any duplicates

In [17]:
print(len(df))
df = df.drop_duplicates(subset=['Node'])
print(len(df))

230
230


Save scaped data as a table in the AO3 mySQL database.

Note: because of the limited data types of mySQL (relative to pandas), the feilds containing lists must be converted strings, and panda's default numpy.floats to integers. 

In [18]:
#def sqlrow(work):

# stringify the lists
import ast
w = df.iloc[1]
col = df.columns
for c in col:
    print(c + ' ' + str(type(w[c])))
    if type(w[c]) is list:
        print(str(w[c]))
        s = str(w[c])
        ll = ast.literal_eval(s)
        print(len(ll))
    if type(w[c]) is np.float64:
        print('int ' + str(int(w[c])))
    if type(w[c]) is str:
        print('str ' + w[c])

Bookmarks <class 'numpy.float64'>
int 3
Category <class 'list'>
['Multi']
1
Chapters <class 'list'>
[44, '?']
2
Character_tags <class 'list'>
['Moira Rose', 'Johnny Rose', 'Alexis Rose', "David Rose (Schitt's Creek)", 'Jazzagals', 'Stevie Budd', 'Jocelyn Schitt', 'Roland Schitt', 'Twyla Sands', 'Bob Currie', 'Ray Butani', 'Ronnie Lee', 'Theodore "Ted" Mullens', 'Gwen Currie', 'Mutt Schitt']
15
Collection <class 'numpy.float64'>
int 0
Comments <class 'numpy.float64'>
int 170
Complete <class 'str'>
str Work in Progress
Creator <class 'list'>
['mrenae']
1
Date <class 'numpy.float64'>
int 20201226
Fandom <class 'list'>
["Schitt's Creek"]
1
Freeform_tags <class 'list'>
['Life-altering news', 'Surprise Baby', 'Family Struggles', 'Comfort', 'Bébé Rose']
5
Gift <class 'list'>
[]
0
Hits <class 'numpy.float64'>
int 2278
Kudos <class 'numpy.float64'>
int 36
Language <class 'str'>
str English
Node <class 'numpy.float64'>
int 27820783
Rating <class 'str'>
str Not Rated
Relationship_tags <class 'lis

In [37]:
os.chdir('/Users/finn/dev/Ao3mine/data')
TODAY = datetime.date.today().isoformat()


In [38]:
dbName = 'test_ronnie_' + str(TODAY)
conn = sqlite3.connect(dbName + '_ao3DB.db')  
c = conn.cursor()

In [49]:
test_df = df.loc[:,['Title','Summary','Bookmarks','Tags']]

c.execute('''DROP TABLE TESTWORKS''')

c.execute('''CREATE TABLE TESTWORKS ([Title] text,[Summary] text,[Bookmarks] integer,[Tags] list)''')

test_df.to_sql('TESTWORKS', conn, if_exists='append', index = False)

InterfaceError: Error binding parameter 3 - probably unsupported type.

In [48]:
test_df = df.loc[:,['Title','Summary','Bookmarks','Tags']]

c.execute('''DROP TABLE TESTWORKS''')

c.execute('''CREATE TABLE TESTWORKS ([Title] text,[Summary] text,[Bookmarks] integer,[Tags] text)''')

test_df.to_sql('TESTWORKS', conn, if_exists='append', index = False)

InterfaceError: Error binding parameter 3 - probably unsupported type.

In [47]:
c.execute("SELECT * FROM TESTWORKS")
data = c.fetchall()
for row in data[:5]:
    print(row)

print(len(data))

('The Rumours are Terrible', "\n“Don’t you think that’s just awful? That he would cheat on his husband with someone called Rose? I mean, he took David’s last name and everything, he works at Rose Apothecary, and he’s having an affair with a woman called Rose.”\n\nJocelyn hears something she just can't ignore, and the town comes together to defend their own.\n", 3)
('Miracles Don’t Take Reservations', 'What if Moira was the one who took the pregnancy test (not Jocelyn)?\n', 3)
('Secret Santa is a Scam', "Patrick and David are coworkers at the headquarters for the chain of RAY's outdoor gear retail stores and have been pining over each other for months. When forced into an office round of secret Santa, Patrick and David draw each other and each consider using it as an opportunity to make their move.\nThis fic will have a daily chapter posted through Christmas, corresponding with the date in the story (and an epilogue on Dec. 26).\n", 26)
('Language of Love: Part 4 (Season 5)', 'This seri

In [ ]:
conn = sqlite3.connect(DBname)
conn.row_factory = sqlite3.Row
cur = conn.cursor()
cur.execute("SELECT * FROM dumptruck")
data = cur.fetchall()
# for row in rows[:5]:
#     print(row)

print(len(data))
print(data[0]['Date'])
print(data[-1]['Date'])

In [39]:

c.execute('''CREATE TABLE WORKS
             ([Bookmarks] integer, [Category] text, [Chapters] text, [Character_tags] text, [Collection] text, [Comments] integer, [Complete] text, [Creator] text, [Date] integer, [Fandom] text, [Freeform_tags] text, [Gift] text, [Hits] integer, [Kudos] integer, [Language] text, [Node] integer, [Rating] text, [Relationship_tags] text, [Series] text, [Series_part] text, [Summary] text, [Tags] text, [Title] text, [Warning] text, [Words] integer, [cURL] text, [sURL] text, [url] text)''')

df.to_sql('WORKS', conn, if_exists='append', index = False) # Insert the values from the csv file into the table 'CLIENTS' 


InterfaceError: Error binding parameter 1 - probably unsupported type.